In [2]:
!pip install rapidfuzz openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.0 MB/s eta 0:00:00


In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from rapidfuzz import fuzz , process
from openpyxl import load_workbook
import time
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from multiprocessing import Pool, cpu_count

In [4]:
seller_sheet=pd.read_excel('/content/Product Matching Dataset.xlsx',sheet_name='Dataset')

In [5]:
master_sheet=pd.read_excel('/content/Product Matching Dataset.xlsx',sheet_name='Master File')

In [6]:
seller_sheet.head()

,sku,marketplace_product_name_ar,seller_item_name,price
0,1322,استوهالت 40 مجم 14 كبسول,ESTOHALT 40 MG 14 CAP,56.5
1,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
2,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
3,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
4,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5


In [7]:
seller_sheet.tail()

,sku,marketplace_product_name_ar,seller_item_name,price
83557,1603,سبانيلا ام آر 35 مجم 30 قرص,سبانيلا 35 مم ق,39.0
83558,1603,سبانيلا ام آر 35 مجم 30 قرص,سبانيلا 35 مل اقراص,39.0
83559,1603,سبانيلا ام آر 35 مجم 30 قرص,سبانيلا 35 مجم م.ر 30 قرص,39.0
83560,1603,سبانيلا ام آر 35 مجم 30 قرص,سبانيلا 35 مجم ام ار اقراص,39.0
83561,1603,سبانيلا ام آر 35 مجم 30 قرص,سبانيلا 35 مجم اقراص س ج,39.0


In [8]:
master_sheet.head()

,sku,product_name,product_name_ar,price
0,279,ANAFRONIL 75 MG 20 TAB,انافرونيل 75 مجم اس ار 20 قرص,75.0
1,2282,LOPRECOUGH SYRUP 100 ML,لوبريكاف شراب 100 مل,28.5
2,4331,TOMEX PLUS 50 TAB,تومكس بلس 50 قرص,60.0
3,1022,TAROLIMUS 0.03% OINT. 15 GM,تاروليمس 0.03 % مرهم 15 جم,129.0
4,116,GLIPTUS PLUS 50/1000 MG 30 TAB,جليبتس بلس 50/1000 مجم 30 قرص,192.0


In [9]:
master_sheet.tail()

,sku,product_name,product_name_ar,price
995,1197,SULFAX PLUS MASSAGE GEL 60 GM,سولفاكس بلس مساج جيل 60 جم,85.0
996,864,MOODAPEX 50 MG 30 TAB,مودابكس 50 مجم 30 قرص,84.0
997,502,KAPRON 500 MG 20 TAB,كابرون 500 مجم 20 قرص,72.0
998,1127,JARDIANCE 10 MG 30 TAB,جارديانس 10 مجم 30 قرص,626.0
999,770,E MOX 500 MG 16 CAP,ايموكس 500 مجم 16 كبسول,40.0


In [10]:
seller_sheet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83562 entries, 0 to 83561
Data columns (total 4 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   sku                          83562 non-null  int64  
 1   marketplace_product_name_ar  83562 non-null  object 
 2   seller_item_name             83562 non-null  object 
 3   price                        83562 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 2.6+ MB


In [11]:
master_sheet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sku              1000 non-null   int64  
 1   product_name     1000 non-null   object 
 2   product_name_ar  1000 non-null   object 
 3   price            1000 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 31.4+ KB


In [12]:
seller_sheet.describe()

,sku,price
count,83562.000000,83562.000000
mean,1600.653204,79.055458
std,1469.206223,62.818117
min,4.000000,7.000000
25%,476.000000,38.000000
50%,1312.000000,61.500000
75%,2258.000000,100.500000
max,9532.000000,406.000000


In [13]:
seller_sheet.shape

(83562, 4)

In [14]:
master_sheet.shape

(1000, 4)

In [15]:
seller_sheet.isnull().sum()

,0
sku,0
marketplace_product_name_ar,0
seller_item_name,0
price,0


In [16]:
seller_sheet.duplicated().sum()

33990

In [17]:
seller_sheet.drop_duplicates(inplace=True)

In [18]:
seller_sheet.duplicated().sum()

0

In [19]:
seller_sheet.shape

(49572, 4)

In [20]:
seller_sheet.head()

,sku,marketplace_product_name_ar,seller_item_name,price
0,1322,استوهالت 40 مجم 14 كبسول,ESTOHALT 40 MG 14 CAP,56.5
1,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
7,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 1 شريط * 14 كبسولة,56.5
8,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم,56.5
9,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 كبسول س ج,56.5


In [21]:
master_sheet.isnull().sum()

,0
sku,0
product_name,0
product_name_ar,0
price,0


In [22]:
master_sheet.duplicated().sum()

0

In [23]:
def clean_text(text):
    text = str(text).lower()
    text = text.replace("س ج", "").strip()
    text = re.sub(r'\bك\b', 'كبسول', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text)

    dosage_forms = {
        'tab': 'tablet',
        'caps': 'capsule',
        'cap': 'capsule',
        'كبسولة': 'كبسول'
    }

    for abbrev, full in dosage_forms.items():
        text = text.replace(abbrev, full)

    return text.strip()


In [24]:
seller_sheet['marketplace_product_name_ar']= seller_sheet['marketplace_product_name_ar'].apply(clean_text)
seller_sheet['seller_item_name']= seller_sheet['seller_item_name'].apply(clean_text)

In [25]:
seller_sheet.head()

,sku,marketplace_product_name_ar,seller_item_name,price
0,1322,استوهالت 40 مجم 14 كبسول,estohalt 40 mg 14 capsule,56.5
1,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 كبسول,56.5
7,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 1 شريط 14 كبسول,56.5
8,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم,56.5
9,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 كبسول,56.5


In [26]:
master_sheet['product_name']= master_sheet['product_name'].apply(clean_text)
master_sheet['product_name_ar'] = master_sheet['product_name_ar'].apply(clean_text)

In [27]:
master_sheet.head()

,sku,product_name,product_name_ar,price
0,279,anafronil 75 mg 20 tablet,انافرونيل 75 مجم اس ار 20 قرص,75.0
1,2282,loprecough syrup 100 ml,لوبريكاف شراب 100 مل,28.5
2,4331,tomex plus 50 tablet,تومكس بلس 50 قرص,60.0
3,1022,tarolimus 003 oint 15 gm,تاروليمس 003 مرهم 15 جم,129.0
4,116,gliptus plus 501000 mg 30 tablet,جليبتس بلس 501000 مجم 30 قرص,192.0


In [28]:
def extract_features(text):
    features = {
        'concentration': None,
        'dosage_form': None
    }
    concentration_match = re.search(r'\d+\s*(mg|مجم|g|جم)', text)
    if concentration_match:
        features['concentration'] = concentration_match.group()

    dosage_forms = ['tab', 'قرص', 'capsule', 'كبسول', 'syrup', 'شراب']
    for form in dosage_forms:
        if form in text:
            features['dosage_form'] = form
            break

    return features

In [29]:
def get_confidence_level(score):
    if score >= 90:
        return "High"
    elif 70 <= score < 90:
        return "Medium"
    else:
        return "Low"

In [42]:
def find_best_match(seller_item_name, master_sheet, lang='ar'):
    start_time = time.time()
    if lang == 'ar':
        query = seller_item_name
        choices = master_sheet['product_name_ar'].tolist()
    else:
        query = seller_item_name
        choices = master_sheet['product_name'].tolist()

    query_features = extract_features(query)

    best_match = None
    best_score = 0

    for choice in choices:

        choice_features = extract_features(choice)


        if query_features['concentration'] == choice_features['concentration'] and \
           query_features['dosage_form'] == choice_features['dosage_form']:
            score = fuzz.token_sort_ratio(query, choice)
            if score > best_score:
                best_score = score
                best_match = choice

    end_time = time.time()
    execution_time = (end_time - start_time) * 1000
    print(f"Time taken for matching: {execution_time:.2f} ms")

    if execution_time > 500:
        print(f"Warning: Matching took {execution_time:.2f} ms, which exceeds the 500 ms limit.")


    similarity_score = best_score / 100

    if best_score >= 90:
        matched_row = master_sheet[master_sheet[f'product_name{"" if lang == "en" else "_ar"}'] == best_match].iloc[0]
        confidence_level = get_confidence_level(best_score)
        return matched_row['sku'], similarity_score, best_match, confidence_level
    else:
        confidence_level = get_confidence_level(best_score)
        return None, similarity_score, best_match, confidence_level
def process_chunk(chunk, master_sheet, lang):
    results = chunk.apply(lambda x: find_best_match(x, master_sheet, lang))
    return results

def parallel_processing(seller_sheet, master_sheet, lang, chunk_size=1000):
    chunks = [seller_sheet[i:i + chunk_size] for i in range(0, seller_sheet.shape[0], chunk_size)]
    with Pool(cpu_count()) as pool:
        results = pool.starmap(process_chunk, [(chunk, master_sheet, lang) for chunk in chunks])
    return pd.concat(results)

In [43]:

seller_sheet['sku_en'], seller_sheet['score_en'], seller_sheet['matched_name_en'], seller_sheet['confidence_en'] = zip(*seller_sheet['seller_item_name'].apply(
    lambda x: find_best_match(x, master_sheet, 'en')
))


seller_sheet['sku_ar'], seller_sheet['score_ar'], seller_sheet['matched_name_ar'], seller_sheet['confidence_ar'] = zip(*seller_sheet['marketplace_product_name_ar'].apply(
    lambda x: find_best_match(x, master_sheet, 'ar')
))

def select_best_match(row):
    if row['score_en'] >= row['score_ar']:
        return row['sku_en'] if row['sku_en'] is not None else row['matched_name_en'], row['score_en'], row['confidence_en']
    elif row['score_ar'] >= row['score_en']:
        return row['sku_ar'] if row['sku_ar'] is not None else row['matched_name_ar'], row['score_ar'], row['confidence_ar']
    else:
        return None, 0, "Low Confidence"

seller_sheet['Expected SKU'], seller_sheet['Similarity Score'], seller_sheet['Confidence Level'] = zip(*seller_sheet.apply(select_best_match, axis=1))

seller_sheet.drop(columns=['sku_en', 'score_en', 'matched_name_en', 'confidence_en', 'sku_ar', 'score_ar', 'matched_name_ar', 'confidence_ar'], inplace=True)

Streaming output truncated to the last 5000 lines.
Time taken for matching: 2.91 ms
Time taken for matching: 2.54 ms
Time taken for matching: 2.55 ms
Time taken for matching: 2.54 ms
Time taken for matching: 2.52 ms
Time taken for matching: 2.55 ms
Time taken for matching: 2.56 ms
Time taken for matching: 2.54 ms
Time taken for matching: 2.66 ms
Time taken for matching: 2.51 ms
Time taken for matching: 2.46 ms
Time taken for matching: 2.56 ms
Time taken for matching: 2.46 ms
Time taken for matching: 2.55 ms
Time taken for matching: 2.52 ms
Time taken for matching: 2.51 ms
Time taken for matching: 2.46 ms
Time taken for matching: 2.56 ms
Time taken for matching: 2.45 ms
Time taken for matching: 2.45 ms
Time taken for matching: 2.50 ms
Time taken for matching: 2.50 ms
Time taken for matching: 2.47 ms
Time taken for matching: 2.55 ms
Time taken for matching: 2.46 ms
Time taken for matching: 2.47 ms
Time taken for matching: 2.49 ms
Time taken for matching: 2.46 ms
Time taken for matching: 

In [44]:
seller_sheet.head()

,sku,marketplace_product_name_ar,seller_item_name,price,Expected SKU,Similarity Score,Confidence Level
0,1322,استوهالت 40 مجم 14 كبسول,estohalt 40 mg 14 capsule,56.5,1322.0,1.0,High
1,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 كبسول,56.5,1322.0,1.0,High
7,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 1 شريط 14 كبسول,56.5,1322.0,1.0,High
8,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم,56.5,1322.0,1.0,High
9,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 كبسول,56.5,1322.0,1.0,High


In [45]:
seller_sheet

,sku,marketplace_product_name_ar,seller_item_name,price,Expected SKU,Similarity Score,Confidence Level
0,1322,استوهالت 40 مجم 14 كبسول,estohalt 40 mg 14 capsule,56.5,1322.0,1.0,High
1,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 كبسول,56.5,1322.0,1.0,High
7,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 1 شريط 14 كبسول,56.5,1322.0,1.0,High
8,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم,56.5,1322.0,1.0,High
9,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 كبسول,56.5,1322.0,1.0,High
...,...,...,...,...,...,...,...
83557,1603,سبانيلا ام آر 35 مجم 30 قرص,سبانيلا 35 مم ق,39.0,1603.0,1.0,High
83558,1603,سبانيلا ام آر 35 مجم 30 قرص,سبانيلا 35 مل اقراص,39.0,1603.0,1.0,High
83559,1603,سبانيلا ام آر 35 مجم 30 قرص,سبانيلا 35 مجم مر 30 قرص,39.0,1603.0,1.0,High
83560,1603,سبانيلا ام آر 35 مجم 30 قرص,سبانيلا 35 مجم ام ار اقراص,39.0,1603.0,1.0,High


In [46]:
if 'sku' in seller_sheet.columns:

    correct_predictions = (seller_sheet['Expected SKU'] == seller_sheet['sku']).sum()

    total_predictions = seller_sheet.shape[0]

    accuracy = (correct_predictions / total_predictions) * 100
    print(f'Accuracy: {accuracy:.2f}%')
else:
    print("Accuracy less than expectation")

Accuracy: 99.58%


In [48]:
seller_sheet.to_excel('matched_products.xlsx', index=False)

Exception ignored in: <function ZipFile.__del__ at 0x7dc17c9ddda0>
Traceback (most recent call last):
  File "/usr/lib/python3.11/zipfile.py", line 1895, in __del__
    self.close()
  File "/usr/lib/python3.11/zipfile.py", line 1912, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file
